In [ ]:
'''
scores
https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0177678
https://machinelearningmastery.com/tour-of-evaluation-metrics-for-imbalanced-classification/
https://machinelearningmastery.com/tour-of-evaluation-metrics-for-imbalanced-classification/
https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-imbalanced-classification/

https://towardsdatascience.com/3-essential-ways-to-calculate-feature-importance-in-python-2f9149592155
'''



| caso | caracterictica | proporcion | acc | 0   | 1   |
| --- | --- | --- | --- | --- | --- |
| df_caso1 | necesitará hospitalización | TIPO_PACIENTE 0:1843528 1:425940 | 74  | 92  | 40  |
| df_caso0 | necesitará hospitalización crítica | hosp_critica 0:350982 1:71582 | 52  | 85  | 20  |
| df_caso3 | Mortalidad DESPUES de INTUBADO,UCI (sin filtro) | BOOL_DEF 0:234136 1:188428 | 67  | 72  | 62  |

In [6]:
import os, sys
import pandas as pd
import numpy as np
import seaborn as sns; sns.set()
from matplotlib import pyplot as plt
from sklearn import metrics
import joblib as joblib
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from imblearn.ensemble import EasyEnsembleClassifier 
from imblearn.ensemble import RUSBoostClassifier
from imblearn.ensemble import BalancedBaggingClassifier 
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.svm import SVC
import shap
from yellowbrick.classifier.rocauc import roc_auc
from yellowbrick.classifier import confusion_matrix
from yellowbrick.features import pca_decomposition
from yellowbrick.classifier import classification_report
import re

In [2]:
path = "/home/nacho/Documents/coronavirus/COVID-19_Paper/"
os.chdir(os.path.join(path)) 

In [3]:
def pred_label(filename): 
    if filename.find('df_caso0') != -1:
        label = 'hosp_critica'
    if filename.find('caso1') != -1:
        label = 'TIPO_PACIENTE'
    if filename.find('caso2') != -1 or filename.find('caso3') != -1 or filename.find('df_caso_3_1') != -1 or filename.find('df_caso_3_2') != -1 or filename.find('df_caso_3_3') != -1:
        label = 'BOOL_DEF'
    if filename.find('caso5') != -1 or filename.find('df_caso5_1') != -1:
        label = 'UCI'
    if filename.find('caso6') != -1 or filename.find('caso7') != -1 or filename.find('df_caso_7_1') != -1 or filename.find('df_caso_7_2') != -1 or filename.find('df_caso_7_3') != -1:
        label = 'INTUBADO'
    return label

In [4]:
dict_clf = {
  "gb": GradientBoostingClassifier(),
  "xgb": XGBClassifier(use_label_encoder=False),
  "eec": EasyEnsembleClassifier(),
  "svc": SVC(),
  "rbc": RUSBoostClassifier(),
  "bbc": BalancedBaggingClassifier(),
  "brf": BalancedRandomForestClassifier()
  }

In [8]:
for subdir, dirs, files in os.walk('models/all_data/20_80'):
    for file in files:
        if file.endswith(".pkl"):
            file_path = subdir + "/" + file
            file_name = file.split('.', 1)[0]
            print(file_name)
            df_data = pd.read_csv("prediction_data/completed_datasets/"+re.sub(r'\D+$', '', file_name)+str(".zip"))
            #df_data = df_data.sample(frac=data_percentage)
            #separar datos
            label = pred_label(file_name)
            print(label)
            X = df_data.loc[:, df_data.columns != label]
            y = df_data.loc[:, label]
            X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,stratify=y,shuffle=True)
            print(y.value_counts())       
            grid = joblib.load(file_path)
            grid_report = pd.read_csv(subdir + "/" + file_name+"_report.csv", index_col=0)
            #report
            df_grid = pd.DataFrame(grid.cv_results_).sort_values(by=['mean_test_ba'],ascending=False)
            df_grid=df_grid.sort_values(by=['mean_test_ba'],ascending=False)
            df_grid=df_grid[[
                'param_clf__selected_model',
                'params',
                'rank_test_ba',
                'mean_fit_time',
                'mean_test_ba',
                'mean_test_F1',
                'mean_test_ra',
                'mean_test_rc',
                'mean_test_ap'
                ]]
            display(df_grid)
            clf = grid.best_params_['clf__selected_model'][0]
            clf_params = grid.best_params_['clf__selected_model'][1]
            clf_params = {clf+"__"+k: v for k, v in clf_params.items()}
            #model
            num_transformer=Pipeline(steps=[
                ('scaler',MinMaxScaler())])
            preprocessor=ColumnTransformer(
                remainder='passthrough',
                transformers=[('num',num_transformer,['EDAD'])])
            model=Pipeline([
                ('preprocessor',preprocessor),
                #('r', SMOTEENN(smote = SMOTE(), enn = EditedNearestNeighbours())),
                (clf, dict_clf[clf])
                ])
            #train
            model.set_params(**clf_params)
            model.fit(X_train,y_train)
            y_pred = model.predict(X_test)
            report = classification_report(y_test, y_pred, output_dict=True)
            report = pd.DataFrame(report).transpose()
            #report
            display(report)
            metrics.roc_auc_score(y_test, y_pred)
            metrics.balanced_accuracy_score(y_test, y_pred)
            metrics.average_precision_score(y_test, y_pred)
            metrics.f1_score(y_test, y_pred)
            #se preprocesan los datos
            X_train_scaled = pd.DataFrame(model.named_steps['preprocessor'].fit_transform(X_train),columns = X_train.columns)
            X_test_scaled = pd.DataFrame(model.named_steps['preprocessor'].fit_transform(X_test),columns = X_test.columns)
            #cambia el orden de edad y sexo
            X_train_scaled[['EDAD','SEXO']]=X_train_scaled[['SEXO','EDAD']]
            X_test_scaled[['EDAD','SEXO']]=X_test_scaled[['SEXO','EDAD']]
            #prueba train
            X_test_scaled.iloc[20, :]
            y_test.iloc[20]
            model.named_steps[clf].predict(X_test_scaled.iloc[20, :].values.reshape(1, -1))
            model.named_steps[clf].predict_proba(X_test_scaled.iloc[20, :].values.reshape(1, -1))
            # use Kernel SHAP to explain test set predictions
            # num_features = len(X_test.columns)
            # explainer = shap.KernelExplainer(model = model.named_steps[clf].predict_proba, data = X_train_scaled, link = 'logit')
            # shap_values=explainer.shap_values(X=X_test_scaled,nsamples=30,l1_reg="num_features(12)")
            # shap.summary_plot(shap_values,X_test)
            # shap.summary_plot(shap_values[1],X_test_scaled
            
            #roc auc curve
            roc_auc(model, X_train, y_train, X_test=X_test, y_test=y_test, classes=['NO '+label, label])
            confusion_matrix(model,X_train,y_train, X_test, y_test, classes=['NO '+label, label])
            pca_decomposition(X, y, scale=True, classes=['NO '+label, label])
            pca_decomposition(X, y, scale=True, classes=['NO '+label, label], projection=3)
            visualizer = classification_report(model, X_train, y_train, X_test, y_test, classes=['NO '+label, label], support=True)


df_caso_7_2_grid
INTUBADO
1    18182
0    16743
Name: INTUBADO, dtype: int64


,param_clf__selected_model,params,rank_test_ba,mean_fit_time,mean_test_ba,mean_test_F1,mean_test_ra,mean_test_rc,mean_test_ap
46,"(brf, {'class_weight': 'balanced_subsample', '...","{'clf__selected_model': ('brf', {'class_weight...",1,40.480043,0.596805,0.660798,0.622581,0.743183,0.600473
57,"(brf, {'class_weight': 'balanced_subsample', '...","{'clf__selected_model': ('brf', {'class_weight...",2,13.417781,0.595970,0.660068,0.622658,0.742335,0.600777
56,"(brf, {'class_weight': 'balanced_subsample', '...","{'clf__selected_model': ('brf', {'class_weight...",3,0.440963,0.595508,0.661682,0.622836,0.748179,0.602510
45,"(brf, {'class_weight': 'balanced', 'criterion'...","{'clf__selected_model': ('brf', {'class_weight...",4,27.098922,0.595055,0.647152,0.624367,0.705465,0.604969
25,"(brf, {'class_weight': 'balanced', 'criterion'...","{'clf__selected_model': ('brf', {'class_weight...",5,15.965682,0.594574,0.646419,0.624129,0.704204,0.604823
42,"(brf, {'class_weight': 'balanced', 'criterion'...","{'clf__selected_model': ('brf', {'class_weight...",6,9.161447,0.594573,0.645755,0.624517,0.702210,0.605262
54,"(brf, {'class_weight': 'balanced', 'criterion'...","{'clf__selected_model': ('brf', {'class_weight...",7,9.265090,0.594401,0.646413,0.624262,0.704456,0.604898
17,"(brf, {'class_weight': 'balanced_subsample', '...","{'clf__selected_model': ('brf', {'class_weight...",8,25.164488,0.594186,0.678717,0.624331,0.805812,0.605142
13,"(brf, {'class_weight': 'balanced_subsample', '...","{'clf__selected_model': ('brf', {'class_weight...",9,28.428445,0.593992,0.678083,0.624446,0.803956,0.605222
31,"(brf, {'class_weight': 'balanced', 'criterion'...","{'clf__selected_model': ('brf', {'class_weight...",10,28.420572,0.593872,0.640410,0.624266,0.688116,0.603692


TypeError: classification_report() missing 1 required positional argument: 'y_train'